In [17]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 4.73MB/s]


In [18]:
!pip install kaggle

In [19]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m


mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 1.44G/1.44G [00:17<00:00, 64.2MB/s]
100% 1.44G/1.44G [00:17<00:00, 86.6MB/s]


In [20]:
!7z e fasttext-crawl-300d-2m.zip -o/content -r



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1545551987 bytes (1474 MiB)

Extracting archive: fasttext-crawl-300d-2m.zip
--
Path = fasttext-crawl-300d-2m.zip
Type = zip
Physical Size = 1545551987

  0%      0% - crawl-300d-2M.vec                          1% - crawl-300d-2M.vec                          2% - crawl-300d-2M.vec                          3% - crawl-300d-2M.vec                          4% - crawl-300d-2M.vec                          5% - crawl-300d-2M.vec                        

In [21]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
# Reading glove vectors in python: https://stackoverflow.com/a/38230349/4084039
def fasttextModel(gloveFile):
    print ("Loading Fasttext Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}#for storing word and the corresponding embedding vector for that word
    for line in f:
        splitLine = line.split()#splitting the line and storing it in a list
        word = splitLine[0]#getting the first element and storing it in word
        embedding = np.array([float(val) for val in splitLine[1:]])#obtaining corresponding vector for that word
        model[word] = embedding#storing word as key and embedding vector for that word as value
    print ("Done.",len(model)," words loaded!")
    return model
model = fasttextModel('/content/crawl-300d-2M.vec')

Loading Fasttext Model
Done. 2000000  words loaded!


In [23]:
df=pd.read_csv('preprocessed_data.csv')#reading data into DataFrame

In [24]:
df.head(4)#displaying top 4 datapoints


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [25]:
def preprocess(x):#removing last character
  x=x[:-1]
  return x

In [26]:
df['source']=df['source'].apply(preprocess)#preprocessing source data
df['target']=df['target'].apply(preprocess)#preprocessing target data

In [27]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [28]:
df.shape#shape of DataFrame


(2000, 2)

In [29]:
df=df[df['source'].apply(len)<170]#removing source sentences of length greater than or equal to 170
df=df[df['target'].apply(len)<200]#removing target sentences of length greater than or equal to 200

In [30]:
df.shape#shape of DataFrame


(1990, 2)

In [31]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting the data in the ratio 99:1
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


In [32]:
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')
X_test.to_csv('X_test.csv')
y_test.to_csv('y_test.csv')


Target:

In [33]:
target_tokenizer= Tokenizer()#tokenization on target
target_tokenizer.fit_on_texts(y_train)#fitting on ytrain
target_vocab_size= len(target_tokenizer.word_index) + 1#target vocab size
print(len(target_tokenizer.word_index))

3033


In [34]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting text to integers
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting text to integers

In [35]:
target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding to maxlength

In [36]:
target_padded_docs_train.shape


(1970, 43)

In [37]:
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [38]:
target_padded_docs_test.shape


(20, 43)

Source:

In [39]:
source_tokenizer= Tokenizer()#tokenization on source
source_tokenizer.fit_on_texts(X_train)#fitting to X_train
source_vocab_size= len(source_tokenizer.word_index) + 1#source vocab size
print(len(source_tokenizer.word_index))

3698


In [40]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting text to sequence
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting text to sequence

In [41]:
source_padded_docs_train = pad_sequences(source_encoded_docs_train,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [42]:
source_padded_docs_train.shape

(1970, 43)

In [43]:
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [44]:
source_padded_docs_test.shape

(20, 43)

In [45]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))

In [46]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [47]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))

In [48]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [49]:
import pandas as pd 
pd.DataFrame(source_encoded_docs_train).to_csv("source_encoded_docs_train.csv")
pd.DataFrame(source_encoded_docs_test).to_csv("source_encoded_docs_test.csv")
pd.DataFrame(target_encoded_docs_train).to_csv("target_encoded_docs_train.csv")
pd.DataFrame(target_encoded_docs_test).to_csv("target_encoded_docs_test.csv")


In [50]:
#creating embedding matrix
embedding_matrix = np.zeros((source_vocab_size, 300))
for word, i in source_tokenizer.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [51]:
embedding_matrix.shape

(3699, 300)

Model1:

In [73]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,300,weights=[embedding_matrix],input_length=source_padded_docs_train.shape[1],trainable=False)(input)
lstm1=tf.keras.layers.LSTM(100, return_sequences=True)(embed)  
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(lstm1)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()                                                                                                                               

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 43, 300)           1109700   
_________________________________________________________________
lstm_4 (LSTM)                (None, 43, 100)           160400    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 43, 3034)          306434    
Total params: 1,576,534
Trainable params: 466,834
Non-trainable params: 1,109,700
_________________________________________________________________


In [74]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [75]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=200,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/200
2/2 [==============================] - 2s 538ms/step - loss: 8.0101 - accuracy: 0.2927 - val_loss: 7.9882 - val_accuracy: 0.6791
Epoch 2/200
2/2 [==============================] - 0s 195ms/step - loss: 7.9829 - accuracy: 0.6694 - val_loss: 7.9436 - val_accuracy: 0.6837
Epoch 3/200
2/2 [==============================] - 0s 192ms/step - loss: 7.9329 - accuracy: 0.6736 - val_loss: 7.8498 - val_accuracy: 0.6837
Epoch 4/200
2/2 [==============================] - 0s 198ms/step - loss: 7.8248 - accuracy: 0.6739 - val_loss: 7.6467 - val_accuracy: 0.6837
Epoch 5/200
2/2 [==============================] - 0s 199ms/step - loss: 7.5876 - accuracy: 0.6739 - val_loss: 7.2696 - val_accuracy: 0.6837
Epoch 6/200
2/2 [==============================] - 0s 192ms/step - loss: 7.1973 - accuracy: 0.6738 - val_loss: 6.8527 - val_accuracy: 0.6837
Epoch 7/200
2/2 [==============================] - 0s 198ms/step - loss: 6.7971 - accuracy: 0.6738 - val_loss: 6.4778 - val_accuracy: 0.6837
Epoch 8/200
2

In [76]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=100,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/100
2/2 [==============================] - 0s 214ms/step - loss: 1.8304 - accuracy: 0.7102 - val_loss: 1.7669 - val_accuracy: 0.7279
Epoch 2/100
2/2 [==============================] - 0s 189ms/step - loss: 1.8268 - accuracy: 0.7111 - val_loss: 1.7647 - val_accuracy: 0.7291
Epoch 3/100
2/2 [==============================] - 0s 192ms/step - loss: 1.8231 - accuracy: 0.7115 - val_loss: 1.7634 - val_accuracy: 0.7302
Epoch 4/100
2/2 [==============================] - 0s 193ms/step - loss: 1.8195 - accuracy: 0.7118 - val_loss: 1.7596 - val_accuracy: 0.7291
Epoch 5/100
2/2 [==============================] - 0s 196ms/step - loss: 1.8160 - accuracy: 0.7124 - val_loss: 1.7578 - val_accuracy: 0.7302
Epoch 6/100
2/2 [==============================] - 0s 195ms/step - loss: 1.8122 - accuracy: 0.7126 - val_loss: 1.7559 - val_accuracy: 0.7302
Epoch 7/100
2/2 [==============================] - 0s 193ms/step - loss: 1.8086 - accuracy: 0.7131 - val_loss: 1.7531 - val_accuracy: 0.7291
Epoch 8/100
2

In [77]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=100,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/100
2/2 [==============================] - 0s 214ms/step - loss: 1.4645 - accuracy: 0.7633 - val_loss: 1.5882 - val_accuracy: 0.7477
Epoch 2/100
2/2 [==============================] - 0s 196ms/step - loss: 1.4612 - accuracy: 0.7640 - val_loss: 1.5913 - val_accuracy: 0.7465
Epoch 3/100
2/2 [==============================] - 0s 192ms/step - loss: 1.4581 - accuracy: 0.7641 - val_loss: 1.5854 - val_accuracy: 0.7477
Epoch 4/100
2/2 [==============================] - 0s 195ms/step - loss: 1.4548 - accuracy: 0.7647 - val_loss: 1.5852 - val_accuracy: 0.7477
Epoch 5/100
2/2 [==============================] - 0s 196ms/step - loss: 1.4512 - accuracy: 0.7654 - val_loss: 1.5841 - val_accuracy: 0.7465
Epoch 6/100
2/2 [==============================] - 0s 194ms/step - loss: 1.4478 - accuracy: 0.7656 - val_loss: 1.5831 - val_accuracy: 0.7465
Epoch 7/100
2/2 [==============================] - 0s 192ms/step - loss: 1.4444 - accuracy: 0.7663 - val_loss: 1.5823 - val_accuracy: 0.7465
Epoch 8/100
2

In [78]:
#https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
#Beam Search
from math import log
from numpy import array
from numpy import argmax
import numpy as np
def beam_search_decoder(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	#print(sequences)
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score - np.log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		sequences = ordered[:k]
	return sequences
 

In [79]:
#prediction
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in x])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output for beam==3 : ")
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  print(' '.join(y_lst1))


  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  print(' '.join(y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  print(' '.join(y_lst3))
  print('>'*180)

Input text: 
Help me collect e clothes, goin to rain....
Actual Output: 
Help me collect the clothes, going to rain.
Predicted Output for beam==3 : 
help me to the and going to to
help me to the going to to
help me to the go going to to
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Mimi40 u now working or studying?
Actual Output: 
Mimi40, are you now working or studying?
Predicted Output for beam==3 : 
i'm you now working or studying
i'll you now working or studying
yes you now working or studying
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
1215 lar... What if i dont have a photo leh? Will they kill me?
Actual Output: 
12:15. What if I don't have a photo? Will they kill me?
Predicted Output fo

In [80]:
import nltk.translate.bleu_score as bleu
bleu_score1=[]
bleu_score2=[]
bleu_score3=[]
#computing bleu_scores for 20 test points where beam=3

for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  bleu_score1.append(bleu.sentence_bleu([b[0].split(),],y_lst1))

  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  bleu_score2.append(bleu.sentence_bleu([b[0].split(),],y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  bleu_score3.append(bleu.sentence_bleu([b[0].split(),],y_lst3))

print("The Average Bleu Score1 is: ",sum(bleu_score1)/20)
print('>'*180)
print("The Average Bleu Score2 is: ",sum(bleu_score2)/20)
print('>'*180)
print("The Average Bleu Score3 is: ",sum(bleu_score3)/20)
print('>'*180)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


The Average Bleu Score1 is:  0.34713550129602855
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score2 is:  0.3496921316896176
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score3 is:  0.3447599549786778
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


Model2:

In [81]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,300,weights=[embedding_matrix],input_length=source_padded_docs_train.shape[1],trainable=False)(input)
lstm1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embed)  
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(lstm1)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 43, 300)           1109700   
_________________________________________________________________
bidirectional (Bidirectional (None, 43, 256)           439296    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 43, 3034)          779738    
Total params: 2,328,734
Trainable params: 1,219,034
Non-trainable params: 1,109,700
_________________________________________________________________


In [82]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [83]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 3s 817ms/step - loss: 7.9019 - accuracy: 0.3301 - val_loss: 6.1246 - val_accuracy: 0.6895
Epoch 2/50
2/2 [==============================] - 0s 240ms/step - loss: 5.4224 - accuracy: 0.6821 - val_loss: 2.8513 - val_accuracy: 0.6837
Epoch 3/50
2/2 [==============================] - 0s 256ms/step - loss: 3.0681 - accuracy: 0.6737 - val_loss: 3.0906 - val_accuracy: 0.6837
Epoch 4/50
2/2 [==============================] - 0s 238ms/step - loss: 3.2525 - accuracy: 0.6737 - val_loss: 2.6719 - val_accuracy: 0.6837
Epoch 5/50
2/2 [==============================] - 0s 243ms/step - loss: 2.7828 - accuracy: 0.6752 - val_loss: 2.3982 - val_accuracy: 0.6907
Epoch 6/50
2/2 [==============================] - 0s 250ms/step - loss: 2.6079 - accuracy: 0.6804 - val_loss: 2.3878 - val_accuracy: 0.6942
Epoch 7/50
2/2 [==============================] - 0s 240ms/step - loss: 2.5251 - accuracy: 0.6820 - val_loss: 2.2288 - val_accuracy: 0.6930
Epoch 8/50
2/2 [====

In [84]:
#https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
#Beam Score
from math import log
from numpy import array
from numpy import argmax
import numpy as np
def beam_search_decoder(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	#print(sequences)
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score - np.log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		sequences = ordered[:k]
	return sequences
 

In [85]:
#prediction
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in x])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output for beam==3 : ")
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  print(' '.join(y_lst1))


  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  print(' '.join(y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  print(' '.join(y_lst3))
  print('>'*180)

Input text: 
Help me collect e clothes, goin to rain....
Actual Output: 
Help me collect the clothes, going to rain.
Predicted Output for beam==3 : 
help me to the and going to
help me to the and going to rain
help me to the to going to
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Mimi40 u now working or studying?
Actual Output: 
Mimi40, are you now working or studying?
Predicted Output for beam==3 : 
i'm you now working or studying
yes you now working or studying
i'm are now working or studying
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
1215 lar... What if i dont have a photo leh? Will they kill me?
Actual Output: 
12:15. What if I don't have a photo? Will they kill me?
Predicted Output for

In [86]:
import nltk.translate.bleu_score as bleu
bleu_score1=[]
bleu_score2=[]
bleu_score3=[]
#computing bleu_scores for 20 test points where beam=3

for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  bleu_score1.append(bleu.sentence_bleu([b[0].split(),],y_lst1))

  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  bleu_score2.append(bleu.sentence_bleu([b[0].split(),],y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  bleu_score3.append(bleu.sentence_bleu([b[0].split(),],y_lst3))

print("The Average Bleu Score1 is: ",sum(bleu_score1)/20)
print('>'*180)
print("The Average Bleu Score2 is: ",sum(bleu_score2)/20)
print('>'*180)
print("The Average Bleu Score3 is: ",sum(bleu_score3)/20)
print('>'*180)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


The Average Bleu Score1 is:  0.3248761532555189
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score2 is:  0.3524630607027792
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score3 is:  0.32389016697371803
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
